In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.ensemble import RandomForestRegressor


In [ ]:
df = pd.read_csv('nyc-rolling-sales.csv')

### Data Exploration and Cleaning

In [ ]:
df.info()

In [ ]:
#Change the types of columns to float and datetime
df['SALE DATE'] = pd.to_datetime(df['SALE DATE'])
df['SALE PRICE'] = pd.to_numeric(df['SALE PRICE'],errors = 'coerce')
df['LAND SQUARE FEET'] = pd.to_numeric(df['LAND SQUARE FEET'],errors = 'coerce')
df['GROSS SQUARE FEET'] = pd.to_numeric(df['GROSS SQUARE FEET'],errors = 'coerce')
df['BOROUGH'] = df['BOROUGH'].astype('category')
df['TAX CLASS AT TIME OF SALE'] = df['TAX CLASS AT TIME OF SALE'].astype('category')


In [ ]:
#remove duplicates
df.drop_duplicates(inplace=True)

In [ ]:
#Check for missing data
df.isnull().sum()

In [ ]:
#Percentage of missing values
df.isnull().sum()/len(df)*100

In [ ]:
#To avoid bias, we can either replace the missing values with the mean or we can remove the rows 
#that contains missing value.


In [ ]:
#Remove of the missing rows
df = df.dropna()
df.isnull().sum()

In [ ]:
#Remove of 'Unnamed' column
df.drop(columns = ['Unnamed: 0'], inplace = True)

In [ ]:
#Check for the statistics of the dataset
df.describe().T

In [ ]:
#There are some min values that seems suspicious. Let's check them!

In [ ]:
#df[df['LAND SQUARE FEET']==0]
#df[df['GROSS SQUARE FEET']==0]
#df[df['YEAR BUILT']==0]
#df[df['SALE PRICE']==0]

In [ ]:
#Remove rows of mins=0 as they interpreted as improper data
df = df[df['LAND SQUARE FEET']!=0]
df = df[df['GROSS SQUARE FEET']!=0]
df = df[df['YEAR BUILT']!=0]
df = df[df['SALE PRICE']!=0]

In [ ]:
#Scatter plot to find outliers
fig, axs = plt.subplots(1, 2, figsize=(10, 8))

sns.scatterplot(data=df, x="YEAR BUILT", y="SALE PRICE",color='red', ax=axs[0])
axs[0].set_title('Year vs Price')
sns.scatterplot(data=df, x="GROSS SQUARE FEET", y="SALE PRICE",color='blue', ax=axs[1])
axs[1].set_title('Gross Square Feet vs Price')

plt.tight_layout()
plt.show()

In [ ]:
#df['YEAR BUILT'].sort_values().head(100)
#df['GROSS SQUARE FEET'].sort_values().head(100)
#df[df['GROSS SQUARE FEET']>100000]

In [ ]:
#We have few outliers. Some of them are very expensive and high gross square feet.
#Need to remove them from our dataset.

df = df[df['YEAR BUILT'] >1848] 
df = df[df['SALE PRICE'] <30000000] 
df=  df[df['GROSS SQUARE FEET']<100000]

In [ ]:
#Checking the plots again
#Scatter plot to find outliers
fig, axs = plt.subplots(1, 2, figsize=(10, 8))

sns.scatterplot(data=df, x="YEAR BUILT", y="SALE PRICE",color='red', ax=axs[0])
axs[0].set_title('Year vs Price')
sns.scatterplot(data=df, x="GROSS SQUARE FEET", y="SALE PRICE",color='blue', ax=axs[1])
axs[1].set_title('Gross Square Feet vs Price')

plt.tight_layout()
plt.show()

In [ ]:
#Checking the statistics again
df.describe().T

In [ ]:
# Even though we removed the zeros, many sales amounts still seems too low to be real.
# We need more limits. We can go with the limits belove
df = df[(df['SALE PRICE']>1000) & (df['SALE PRICE']<6000000)]


In [ ]:
#Sale Prices of the properties
plt.figure(figsize = (15,10))
sns.histplot(df['SALE PRICE'])


In [ ]:
np.quantile(df['SALE PRICE'], .75)


In [ ]:
#More than 75% of properties are worth less than $1 million.
#Again, we observe that a significant part of it is close to 0.

In [ ]:
#Brough Vs Sale Price
plt.figure(figsize=(10,6))
sns.boxplot(x='BOROUGH', y="SALE PRICE", data=df,showmeans=True)
plt.title('BOROUGH vs Sale Price')

In [ ]:
#In this boxplot we can see the means and medians of the boroughs.
#Borough 1 seems to be the most expensive.

In [ ]:
#Trendline for Sales Prices throughout the year
plt.figure(figsize=(10,6))
sns.lineplot(x=df['SALE DATE'].dt.month_name(), y="SALE PRICE", data=df)
plt.title('Trend of Sales by Months')

In [ ]:
#We see that house prices are highest in the summer months, followed by the spring months,
#and lowest in the winter months.

In [ ]:
#correlation matrix
f, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(df.corr(), cmap = 'coolwarm', annot= True)
plt.title('Variable Correlation')

In [ ]:
# We need this correlations to select the features of the model that we will build.
df.corr()['SALE PRICE'].sort_values(ascending=False)

In [ ]:
#We can see correlation between 'Sale Price and Gross Square Feet' and 'Tax Class At Time Of Sale'.
#There are week correlation between 'Sale Price and Residential Units'

In [ ]:
#We can say that 'Gross Square Feet' and 'TAX CLASS AT TIME OF SALE' affect the price most

# Regression Model

In [ ]:
x=df[['GROSS SQUARE FEET','BOROUGH','TAX CLASS AT TIME OF SALE','RESIDENTIAL UNITS','LOT']]

In [ ]:
y = df['SALE PRICE']

### One-Hot Encoding

In [ ]:
#Select the variables to be one-hot encoded
one_hot_features = ['BOROUGH','TAX CLASS AT TIME OF SALE']

one_hot_encoded = pd.get_dummies(x[one_hot_features])
one_hot_encoded.info(verbose=True, memory_usage=True)

In [ ]:
# Replacing categorical columns with dummies
x = x.drop(one_hot_features,axis=1)
x = pd.concat([x, one_hot_encoded] ,axis=1)

### Model Training

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = .20, random_state= 0)

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(x_train,y_train)

In [ ]:
y_pred = regressor.predict(x_test)

### Plot of the test group and pred group

In [ ]:
sns.histplot(data=y_test - y_pred, bins=50, kde=True)


### Performance Of The Model

In [ ]:
from sklearn import metrics
print('MAE:',metrics.mean_absolute_error(y_test,y_pred))
print('MSE:',metrics.mean_squared_error(y_test,y_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test,y_pred)))

In [ ]:
#MAE is not very sensitive to outliers in comparison to MSE since it doesn’t punish huge errors.
#MSE is more accurate than MAE as it highlights large errors over small ones.
#RMSE assigns a higher weight to larger errors as errors are squared before they are averaged.

In [ ]:
#We know there are outliers in the dataset even though we removed some of them. So MAE is the best fit for our model.

In [ ]:
r2 = metrics.r2_score(y_test,y_pred)
print(r2)

In [ ]:
#R-squared is the “percent of variance explained” by the model. 
#R2=0.24, it means that 24% of the variance in the dependent variable can be explained by the independent variable

# Bonus Model To Discuss: Random Forest Regression

In [ ]:
rf_regr = RandomForestRegressor()
rf_regr.fit(x_train, y_train)


In [ ]:
y_pred_rf = rf_regr.predict(x_test)


In [ ]:
print('MAE:',metrics.mean_absolute_error(y_test,y_pred_rf))
print('MSE:',metrics.mean_squared_error(y_test,y_pred_rf))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test,y_pred_rf)))

In [ ]:
r2 = metrics.r2_score(y_test,y_pred_rf)
print(r2)